# Saving Private Prati

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import glob as glob
%matplotlib widget

In [2]:
import pandas as pd

In [3]:
import sep

In [4]:
from photutils.centroids import centroid_com

In [5]:
from scipy.optimize import curve_fit

In [6]:
import seaborn as sns

In [7]:
# Define the Gaussian function
def gaussian(x, amplitude, mean, sigma):
    return amplitude * np.exp(-(x - mean)**2 / (2 * sigma**2))

In [8]:

def gaussian_psf(size, sigma, center=None):
    """
    Generate a 2D Gaussian PSF.
    :param size: Tuple of the shape of the image (height, width)
    :param sigma: Standard deviation of the Gaussian PSF
    :param center: Tuple for the center of the PSF (y, x). If None, it will be at the center of the image.
    :return: 2D numpy array representing the PSF
    """
    y, x = np.indices(size)
    
    if center is None:
        center_y, center_x = size[0] // 2, size[1] // 2
    else:
        center_y, center_x = center
    
    gauss = np.exp(-((x - center_x)**2 + (y - center_y)**2) / (2 * sigma**2))
    return gauss / gauss.max()  # Normalize to have max value of 1


In [9]:
list  = glob.glob("/home/romain/Bureau/0mm/*tiff")

In [ ]:
list

In [30]:
cube = np.zeros((len(list),2046,2048))
cpt=0
for img in list :
    img_arr = np.array(Image.open(img))
    cube[cpt,:,:] = img_arr
    cpt += 1

In [33]:
cube_mean = np.median(cube,axis=0)#[800:][:]

In [ ]:
fig = plt.figure()
plt.imshow(cube_mean, cmap='hot')
plt.show()

In [15]:
cube_mean = cube_mean/np.max(cube_mean)

In [ ]:
np.shape(cube_mean)

In [ ]:
fig = plt.figure()
plt.imshow(cube_mean, cmap='hot')
plt.show()

In [18]:
roi = 200

In [19]:
size = np.shape(cube_mean)

In [20]:
img = cube_mean

In [21]:
flux_ceil = 0.8

data = sep.extract(cube_mean,flux_ceil)
df = pd.DataFrame(data)

x,y = df[["x","y"]].values[0][0],df[["x","y"]].values[0][1]

x,y = round(x), round(y)

cut_x = img[y][int(size[1]/2 - roi/2) :int(size[1]/2 +roi/2)]
cut_y = img[x, int(size[0]/2 - roi/2) :int(size[0]/2 +roi/2)]

# Generate the corresponding x indices for the length of cut_x
x_data = np.arange(len(cut_x))
y_data = np.arange(len(cut_y))

# Fit the Gaussian curve
params_x, covariance_x = curve_fit(gaussian, x_data, cut_x, p0=[np.max(cut_x), np.mean(x_data), np.std(x_data)])
params_y, covariance_y = curve_fit(gaussian, y_data, cut_y, p0=[np.max(cut_y), np.mean(y_data), np.std(y_data)])

# Extract the parameters
amplitude_x, mean_x, sigma_x = params_x
amplitude_y, mean_y, sigma_y = params_y

# Create a DataFrame with the parameters
df_x = pd.DataFrame({
    'Amplitude_x': [amplitude_x],
    'Mean_x': [mean_x],
    'Sigma_x': [sigma_x]
})

df_y = pd.DataFrame({
    'Amplitude_y': [amplitude_y],
    'Mean_y': [mean_y],
    'Sigma_y': [sigma_y]
})

df_gauss = pd.concat([df_x,df_y],axis=1)

df_sep_fwhm = pd.concat([df,df_gauss],axis=1)

In [ ]:
fig = plt.figure()
plt.imshow(cube_mean[y-int(roi/2):y+int(roi/2),x-int(roi/2):x+int(roi/2)], cmap='hot')
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(x_data,cut_x)
plt.show()

In [ ]:
y_data

In [ ]:
cut_x